In [1]:
import sys
sys.path.insert(0, '..')
from mlconfound.stats import confound_test
from mlconfound.simulate import simulate_cyyhat
from mlconfound.plot import plot_null_dist, plot_graph

import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ImportError: cannot import name 'simulate' from 'mlconfound.simulate' (../mlconfound/simulate.py)

# Simulate some data for the null and alternative hypotheses

In [ ]:
#                           true signal       confounder
H0_y, H0_yhat, H0_c = simulate(0.6,              0.3,     # y
                               0.0,              0.9,     # c
                               0.4,              0.2,     # yhat
                               n=500,                     # simulated sample size
                               random_state=42
                               )

#                           true signal       confounder
H1_y, H1_yhat, H1_c = simulate(0.6,              0.3,     # y
                               0.0,              0.9,     # c
                               0.3,              0.2,     # yhat
                               n=500,                     # simulated sample size
                               random_state=42
                              )

# Do the tests for H0 simulation

In [ ]:
ret=confound_test(H0_y, H0_yhat, H0_c, num_perms=1000, return_null_dist=True, 
                  random_state=42)
#print results
pd.DataFrame({
    'p' : [ret.p],
    'ci lower' : [ret.p_ci[0]],
    'ci upper' : [ret.p_ci[1]],
    'R2(y,c)' : [ret.r2_y_c],
    'R2(y^,c)' : [ret.r2_yhat_c],
    'R2(y,y^)' : [ret.r2_y_yhat],
})


## Use the built-in plot functions:

In [ ]:
plot_null_dist(ret)
plot_graph(ret)

# Do the tests for H1 simulation

In [ ]:
ret=confound_test(H1_y, H1_yhat, H1_c, num_perms=1000, return_null_dist=True, 
                  random_state=42, n_jobs=-1)

#print results
pd.DataFrame({
    'p' : [ret.p],
    'ci lower' : [ret.p_ci[0]],
    'ci upper' : [ret.p_ci[1]],
    'R2(y,c)' : [ret.r2_y_c],
    'R2(y^,c)' : [ret.r2_yhat_c],
    'R2(y,y^)' : [ret.r2_y_yhat],
})

In [ ]:
plot_null_dist(ret)
plot_graph(ret, y_name='pain sensitivity', yhat_name='prediction', c_name='center', outfile_base='example')

In [1]:
import numpy as np
from fcit import fcit

# Generate some data such that x is indpendent of y given z.
n_samples = 1000
z = np.random.dirichlet(alpha=np.ones(2), size=n_samples)
x = np.vstack([np.random.multinomial(20, p) for p in z]).astype(float)
y = np.vstack([np.random.multinomial(20, p) for p in z]).astype(float)

# Check that x and y are dependent (p-value should be uniform on [0, 1]).
pval_d = fcit.test(x, y)
# Check that z d-separates x and y (the p-value should be small).
pval_i = fcit.test(x, y, z)

In [ ]:
sns.scatterplot(y[0], x[0])

In [ ]:
confound_test(x,y,z, n_jobs=-1)